In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_type_counts[application_type_counts < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\19194\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Attempt # 1 

## Drop noisy features, use 2 hidden layers (80 & 30 neurons) 

In [10]:
print(len(application_df.columns))
application_df.columns

41


Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
 

In [11]:
# Drop USE_CASE_Other and AFFILIATION_Other columns
temp_application_df= application_df.drop(columns= ["USE_CASE_Other","AFFILIATION_Other"])
temp_application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split the preprocessed data 
y=temp_application_df["IS_SUCCESSFUL"].values
X=temp_application_df.drop(columns=["IS_SUCCESSFUL"]).values

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=.2, random_state= 3)

In [13]:
# Create StandArdScaler 
scaler= StandardScaler()

# Fit the StandardScaler
X_scaler= scaler.fit(X_train)

# Scale the data 
X_train_scaled= X_scaler.transform(X_train)
X_test_scaled= X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3120      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9,681
Trainable params: 9,681
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', period=5)

In [17]:
# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=200, verbose=1, callbacks=[cp_callback])

Epoch 1/200
858/858 [==============================] - 2s 1ms/step - loss: 0.5714 - accuracy: 0.7233
Epoch 2/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7291
Epoch 3/200
858/858 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7304
Epoch 4/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7310
Epoch 5/200
845/858 [============================>.] - ETA: 0s - loss: 0.5539 - accuracy: 0.7303
Epoch 5: saving model to checkpoints\weights.05.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 6/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7320
Epoch 7/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5521 - accuracy: 0.7302
Epoch 8/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7329
Epoch 9/200
858/858 [==============================] -

840/858 [============================>.] - ETA: 0s - loss: 0.5413 - accuracy: 0.7380
Epoch 65: saving model to checkpoints\weights.65.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5403 - accuracy: 0.7384
Epoch 66/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7378
Epoch 67/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7383
Epoch 68/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7384
Epoch 69/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5403 - accuracy: 0.7383
Epoch 70/200
858/858 [==============================] - ETA: 0s - loss: 0.5406 - accuracy: 0.7381
Epoch 70: saving model to checkpoints\weights.70.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7381
Epoch 71/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7389
Epoch 72/200
858/858 

858/858 [==============================] - 2s 2ms/step - loss: 0.5389 - accuracy: 0.7392
Epoch 128/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7388
Epoch 129/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7389
Epoch 130/200
837/858 [============================>.] - ETA: 0s - loss: 0.5385 - accuracy: 0.7383
Epoch 130: saving model to checkpoints\weights.130.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5381 - accuracy: 0.7390
Epoch 131/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5392 - accuracy: 0.7389
Epoch 132/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7394
Epoch 133/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7392
Epoch 134/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5382 - accuracy: 0.7389
Epoch 135/200
843/858 [==========================

834/858 [============================>.] - ETA: 0s - loss: 0.5374 - accuracy: 0.7387
Epoch 190: saving model to checkpoints\weights.190.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5370 - accuracy: 0.7393
Epoch 191/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5371 - accuracy: 0.7397
Epoch 192/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5380 - accuracy: 0.7397
Epoch 193/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5391 - accuracy: 0.7397
Epoch 194/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5370 - accuracy: 0.7391
Epoch 195/200
832/858 [============================>.] - ETA: 0s - loss: 0.5376 - accuracy: 0.7394
Epoch 195: saving model to checkpoints\weights.195.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5375 - accuracy: 0.7393
Epoch 196/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5372 - accuracy: 0.7402
Epoch 197/2

In [18]:
#Evaluate results
model_loss, model_accuracy= nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss:{model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5855 - accuracy: 0.7293 - 703ms/epoch - 3ms/step
Loss:0.5855242013931274, Accuracy: 0.729300320148468


# Attemp # 2

## Use 3 hidden layers (80, 30, and 15 neurons)

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 30
hidden_node3= 15

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="relu"))
# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="relu"))

# Third Hidden Layer
nn2.add(tf.keras.layers.Dense(units= hidden_node3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3120      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 15)                1215      
                                                                 
 dense_6 (Dense)             (None, 1)                 16        
                                                                 
Total params: 10,831
Trainable params: 10,831
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model 
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', period=5)

In [22]:
# Train the model
fit_model= nn2.fit(X_train_scaled, y_train, epochs=200, verbose=1, callbacks=[cp_callback])

Epoch 1/200
858/858 [==============================] - 2s 1ms/step - loss: 0.5698 - accuracy: 0.7219
Epoch 2/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5577 - accuracy: 0.7293
Epoch 3/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5552 - accuracy: 0.7313
Epoch 4/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7325
Epoch 5/200
850/858 [============================>.] - ETA: 0s - loss: 0.5527 - accuracy: 0.7331
Epoch 5: saving model to checkpoints\weights.05.hdf5
858/858 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7332
Epoch 6/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7329
Epoch 7/200
858/858 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7327
Epoch 8/200
858/858 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7334
Epoch 9/200
858/858 [==============================] -

847/858 [============================>.] - ETA: 0s - loss: 0.5402 - accuracy: 0.7393
Epoch 65: saving model to checkpoints\weights.65.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7389
Epoch 66/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5399 - accuracy: 0.7388
Epoch 67/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7382
Epoch 68/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 0.7388
Epoch 69/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5399 - accuracy: 0.7385
Epoch 70/200
845/858 [============================>.] - ETA: 0s - loss: 0.5397 - accuracy: 0.7388
Epoch 70: saving model to checkpoints\weights.70.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5399 - accuracy: 0.7384
Epoch 71/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5393 - accuracy: 0.7384
Epoch 72/200
858/858 

858/858 [==============================] - 1s 2ms/step - loss: 0.5381 - accuracy: 0.7391
Epoch 128/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7394
Epoch 129/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5376 - accuracy: 0.7402
Epoch 130/200
851/858 [============================>.] - ETA: 0s - loss: 0.5370 - accuracy: 0.7402
Epoch 130: saving model to checkpoints\weights.130.hdf5
858/858 [==============================] - 1s 2ms/step - loss: 0.5373 - accuracy: 0.7398
Epoch 131/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5372 - accuracy: 0.7392
Epoch 132/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5371 - accuracy: 0.7399
Epoch 133/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5372 - accuracy: 0.7398
Epoch 134/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5374 - accuracy: 0.7402
Epoch 135/200
837/858 [==========================

858/858 [==============================] - ETA: 0s - loss: 0.5367 - accuracy: 0.7397
Epoch 190: saving model to checkpoints\weights.190.hdf5
858/858 [==============================] - 1s 2ms/step - loss: 0.5367 - accuracy: 0.7397
Epoch 191/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5367 - accuracy: 0.7401
Epoch 192/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5377 - accuracy: 0.7400
Epoch 193/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7398
Epoch 194/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5363 - accuracy: 0.7403
Epoch 195/200
843/858 [============================>.] - ETA: 0s - loss: 0.5368 - accuracy: 0.7397
Epoch 195: saving model to checkpoints\weights.195.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7403
Epoch 196/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7404
Epoch 197/2

In [23]:
# Evaluate Results
model_loss, model_accuracy= nn2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss:{model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5873 - accuracy: 0.7287 - 351ms/epoch - 2ms/step
Loss:0.5873368382453918, Accuracy: 0.7287172079086304


# Attempt # 3

## Use new activation function 'tanh'

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 30
hidden_node3 = 15

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="tanh"))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="tanh"))

# Third Hidden Layer
nn3.add(tf.keras.layers.Dense(units= hidden_node3, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3120      
                                                                 
 dense_8 (Dense)             (None, 80)                6480      
                                                                 
 dense_9 (Dense)             (None, 15)                1215      
                                                                 
 dense_10 (Dense)            (None, 1)                 16        
                                                                 
Total params: 10,831
Trainable params: 10,831
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', period=5)

In [27]:
# Train the model
fit_model= nn3.fit(X_train_scaled, y_train, epochs=200, verbose=1, callbacks=[cp_callback])

Epoch 1/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5711 - accuracy: 0.7224
Epoch 2/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7278
Epoch 3/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5568 - accuracy: 0.7297
Epoch 4/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7321
Epoch 5/200
835/858 [============================>.] - ETA: 0s - loss: 0.5522 - accuracy: 0.7330
Epoch 5: saving model to checkpoints\weights.05.hdf5
858/858 [==============================] - 1s 2ms/step - loss: 0.5526 - accuracy: 0.7322
Epoch 6/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5515 - accuracy: 0.7311
Epoch 7/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7321
Epoch 8/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7331
Epoch 9/200
858/858 [==============================] -

857/858 [============================>.] - ETA: 0s - loss: 0.5391 - accuracy: 0.7395
Epoch 65: saving model to checkpoints\weights.65.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7396
Epoch 66/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5392 - accuracy: 0.7391
Epoch 67/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7396
Epoch 68/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5388 - accuracy: 0.7395
Epoch 69/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5386 - accuracy: 0.7393
Epoch 70/200
852/858 [============================>.] - ETA: 0s - loss: 0.5389 - accuracy: 0.7390
Epoch 70: saving model to checkpoints\weights.70.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5389 - accuracy: 0.7391
Epoch 71/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7394
Epoch 72/200
858/858 

858/858 [==============================] - 2s 2ms/step - loss: 0.5368 - accuracy: 0.7394
Epoch 128/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7405
Epoch 129/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7402
Epoch 130/200
832/858 [============================>.] - ETA: 0s - loss: 0.5364 - accuracy: 0.7402
Epoch 130: saving model to checkpoints\weights.130.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5368 - accuracy: 0.7396
Epoch 131/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5366 - accuracy: 0.7399
Epoch 132/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5371 - accuracy: 0.7404
Epoch 133/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5371 - accuracy: 0.7402
Epoch 134/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5366 - accuracy: 0.7399
Epoch 135/200
846/858 [==========================

842/858 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7405
Epoch 190: saving model to checkpoints\weights.190.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7406
Epoch 191/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5358 - accuracy: 0.7410
Epoch 192/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7404
Epoch 193/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7399
Epoch 194/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5360 - accuracy: 0.7402
Epoch 195/200
849/858 [============================>.] - ETA: 0s - loss: 0.5366 - accuracy: 0.7398
Epoch 195: saving model to checkpoints\weights.195.hdf5
858/858 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7402
Epoch 196/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7405
Epoch 197/2

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5647 - accuracy: 0.7276 - 314ms/epoch - 1ms/step
Loss: 0.5647318363189697, Accuracy: 0.7275510430335999


In [29]:
# Export the model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimization.h5")